# Conditional RAG Cookbook

In [2]:
# setup Arize Phoenix for logging/observability
import phoenix as px

px.launch_app()
import llama_index

llama_index.set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit http://127.0.0.1:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


## Data

In [3]:
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt' -O pg_essay.txt

--2024-02-08 18:06:00--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘pg_essay.txt’

pg_essay.txt        100%[===================>]  73.28K  --.-KB/s    in 0.01s   

2024-02-08 18:06:00 (6.68 MB/s) - ‘pg_essay.txt’ saved [75042/75042]



In [5]:
from llama_index import SimpleDirectoryReader

reader = SimpleDirectoryReader("../data/paul_graham")
docs = reader.load_data()

## Build Components

In [7]:
from llama_index import (
    VectorStoreIndex,
    SummaryIndex,
    load_index_from_storage,
)

In [8]:
import os
from llama_index.storage import StorageContext

if not os.path.exists("storage"):
    index = VectorStoreIndex.from_documents(docs)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage")
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")


summary_index = SummaryIndex.from_documents(docs)

## Demonstrate Routing 

In [1]:
from llama_index.query_pipeline import QueryPipeline, InputComponent, Link, ConditionalLinks, FunctionComponent
from llama_index.selectors import LLMSingleSelector
from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate

ImportError: cannot import name 'ConditionalLinks' from 'llama_index.query_pipeline' (/Users/jerryliu/Programming/gpt_index/llama_index/query_pipeline/__init__.py)

In [ ]:
choices = [
    "This tool answers specific questions about the document (not summary questions across the document)",
    "This tool answers summary questions about the document (not specific questions)",
]

def select_choice(query: str) -> Dict:
    selector = LLMSingleSelector.from_defaults()
    output = selector.select(choices, query)
    return {"query": query, "index": output.ind}

qp = QueryPipeline(modules={
    "input": InputComponent(),
    "selector": FunctionComponent(fn=select_choice),
    "vector_query_engine": vector_query_engine,
    "summary_query_engine": summary_query_engine
})

qp.add_link("input", "selector")
qp.add_conditional_links(
    "selector", 
    lambda x: (x["index"], x["query"]),
    {
        0: vector_query_engine,
        1: summary_query_engine
    }   
)